In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.decomposition import NMF
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE

In [3]:
def graph_3d(data: pd.DataFrame, prediction: pd.Series, name: dict=None, axes: tuple=(0,1,2), opacity: dict=None,):
    """Documentation
    Parameters:
        data : the data that will be show on the plot
        prediction : the prediction that will be used to make distinction bewteen the differents class on the plot
        name : clusters names (ex: {prediction_value: name})
        axes : axes of pca to display points
        opacity : the transparency of the points on the graph
    """

    if opacity is None:
        opacity: dict = {val:1 for val in set(prediction)}
    
    if name is None:
        name: dict = {val:str(val) for val in set(prediction)}

    sc: StandardScaler = StandardScaler()
    X: np.ndarray = sc.fit_transform(data)

    pca: PCA = PCA(n_components=10) # ACP très rapide 
    # pca = TSNE(n_components=3) # TSNE Très très long
    X: np.ndarray = pca.fit_transform(X)

    data_plot: pd.DataFrame = pd.DataFrame(X)
    data_plot["prediction"] = prediction
    # data_plot["prediction"] = prediction.replace([-1,1], name)

    data: list = []
    for val in set(prediction):
        data_semi_plot: pd.DataFrame = data_plot[data_plot["prediction"] == val]
        data.append(go.Scatter3d(x=data_semi_plot[axes[0]], y=data_semi_plot[axes[1]], z=data_semi_plot[axes[2]], name=name[val], mode='markers',marker=dict(size=6), opacity=opacity[val]))

    fig: go.Figure = go.Figure(data)
    fig.show()




In [4]:
bow_g3: pd.DataFrame = pd.read_json("/content/drive/MyDrive/G5 Inter-Promo 2021/Données/Input/g3_BOW_v1.json")
bow_g3.drop(["art_id", "art_content_clean_without_lem"], axis=1, inplace=True)

In [11]:
#for label in np.unique(ms_pred):
print (len(np.unique(ms_pred)))

5720


In [9]:
from sklearn.cluster import MeanShift, estimate_bandwidth
ms:MeanShift = MeanShift(bandwidth = 2 , seeds=None, bin_seeding=False, min_bin_freq=1, cluster_all=True, n_jobs=None, max_iter=50 )
ms.fit(bow_g3)
ms_pred = ms.predict(bow_g3)
graph_3d(bow_g3, ms_pred)
pd.Series(ms_pred).value_counts()

# from sklearn.cluster import KMeans
# kmeans = KMeans(11, random_state=0).fit(bow_g3)
# kmeans_pred = kmeans.predict(bow_g3)
# graph_3d(bow_g3, kmeans_pred)
# pd.Series(kmeans_pred).value_counts()

0       1685
1          8
4376       6
2          6
3820       4
        ... 
1258       1
3307       1
5356       1
1262       1
2045       1
Length: 5720, dtype: int64